# COS 526, Fall 2018
#Point Cloud Registration
#Kara Bressler

In [1]:
import numpy as np
from io import StringIO
import math
from pathlib import Path
from scipy.spatial import KDTree

NUM_SAMPLES = 1000;

# USER INPUT
file1Name = "top3";
file2Name = "bun000";

In [2]:
# 1: PRELIMINARY CODE INFRASTRUCTURE
# Read a point cloud from a .pts file and store this in two NumPy ndarrays, 
# one ndarray for the coordinates of the point and the other for the normals.
# 
# The format is plain ASCII, and looks like the following:
# coord1_x coord1_y coord1_z norm1_x norm1_y norm1_z
# coord2_x coord2_y coord2_z norm2_x norm2_y norm2_z
# ...
# coordn_x coordn_y coordn_z normn_x normn_y normn_z
# Each line just specifies the (x,y,z) coordinates of a point together with its normal.
#
# Referenced File Handling Cheat Sheet for Python (https://www.pythonforbeginners.com/cheatsheet/python-file-handling)

def readPointCloud(_fileName):
    fileName   = _fileName;
    fileLength = len(open(fileName, 'r').readlines(  ));
    file       = open(fileName, 'r');
    
    pointMatrix = np.ndarray( (fileLength, 6), float );

    for lineIndex, line in enumerate(file):
        lineElements = line.split();

        for elementIndex, element in enumerate(lineElements):  
            pointMatrix[lineIndex][elementIndex] = element;
            
    file.close();

    return pointMatrix


In [3]:
# 2: PRELIMINARY CODE INFRASTRUCTURE
# Read a rigid-body transformation, in a simple ASCII format 
# that specifies it as a 4×4 matrix in homogeneous coordinates. 
# 
# For example, a transformation that rotates by 90 degrees counterclockwise around the Z axis, 
# and translates by (2,3,4), would be written as:
#
#    0 -1  0  2
#    1  0  0  3
#    0  0  1  4
#    0  0  0  1
# 
# Referenced COS 426 notes (http://www.cs.princeton.edu/courses/archive/spr17/cos426/notes/cos426_s17_lecture10_transform.pdf) 
# concerning transformations in homogeneous coordinates.

def readTransformation(_fileName):
    fileName = _fileName;
    pathName = Path(fileName);
    
    # If .xf file is missing, assume the identity transformation, as written in the bun000.xf file.
    if not pathName.exists():
        fileName = "bun000.xf";
        
    file = open(fileName, 'r');
    
    transformationMatrix = np.ndarray( (4, 4), float );

    for lineIndex, line in enumerate(file):
        lineElements = line.split();
        for elementIndex, element in enumerate(lineElements):         
            transformationMatrix[lineIndex][elementIndex] = element;

    file.close();

    return transformationMatrix;


In [4]:
# 3: PRELIMINARY CODE INFRASTRUCTURE
# Given an arbitrary point in 3D space, find the closest point in a point set. 
# Here we start with a brute-force implementation:

# Measure distance between two points
def ptDistance(pt1, pt2): 
    distance = math.sqrt( ( (pt1[0] - pt2[0])**2 ) + \
                          ( (pt1[1] - pt2[1])**2 ) + \
                          ( (pt1[2] - pt2[2])**2 ) );
    return distance;

def findClosestPointBruteForce(referencePt, ptSet): 
    shortestDistance = ptDistance(referencePt, ptSet[0]);
    closestPt = ptSet[0];
    
    for pt in ptSet: 
        # Measure distance from current point to reference point
        currentDistance = ptDistance(pt, referencePt);
        if currentDistance < shortestDistance: 
            closestPt = pt;
            shortestDistance = currentDistance;
            
    return closestPt;

# Here we implement an accelerated version based on a [grid] or a [k-d tree].
# Referenced COS 226 notes (http://www.cs.princeton.edu/courses/archive/fall15/cos226/lectures/99GeometricSearch.pdf)
# concerning kd-trees as well as StackOverflow to understand code for implementing kd-trees (https://stackoverflow.com/questions/2486093/millions-of-3d-points-how-to-find-the-10-of-them-closest-to-a-given-point)
# regarding SciPy's KDTree class (https://docs.scipy.org/doc/scipy/reference/spatial.html#spatial-data-structures-and-algorithms).
def findClosestPointKDTree(referencePt, ptSet):
    
    # find 10 nearest points
    threeDimensionaltree = KDTree( ptSet, leafsize=len(ptSet) )
    distances, index = threeDimensionaltree.query( [referencePt], k=1 )
    
    closestPt = ptSet[index]
    
    return closestPt;

In [5]:
# 4: PRELIMINARY CODE INFRASTRUCTURE
# Apply a rigid-body transformation to a point. 
# Applying a transformation to a point is just a matrix multiplication by a column vector:

def transformPt(transformationMatrix, pt):
    return np.dot(transformationMatrix, pt)


In [6]:
# 5: PRELIMINARY CODE INFRASTRUCTURE
# Compose (multiply) two transformations.
def multiplyMatrices(aMatrix, bMatrix):
    return np.dot(aMatrix, bMatrix)

# Invert a given rigid-body transform. 
# The inverse of a rotation matrix is just its transpose. 
# Still need to compute the correct translation for the inverse.
# Consulted Stanford notes to understand inverse matrices (http://www.graphics.stanford.edu/courses/cs248-98-fall/Final/q4.html)
def invertTransformationMatrix(transformationMatrix):
    
    inverseRotationMatrix = np.ndarray( (4, 4), float );
    inverseRotationMatrix[3][3] = 1;
    for index in range(0, 3): 
        inverseRotationMatrix[index][3] = 0;
        inverseRotationMatrix[3][index] = 0;
    
    for x in range(0, 3): 
        for y in range(0, 3): 
            inverseRotationMatrix[y][x] = transformationMatrix[x][y];
    
    inverseTranslationMatrix = np.ndarray( (4, 4), float );
    inverseTranslationMatrix[3][3] = 1;
    
    for x in range(0, 4): 
        for y in range(0, 4): 
            if x == y: 
                inverseTranslationMatrix[x][y] = 1; 
            else: 
                inverseTranslationMatrix[x][y] = 0;
    
    inverseTranslationMatrix[0][3] = -transformationMatrix[0][3];
    inverseTranslationMatrix[1][3] = -transformationMatrix[1][3];
    inverseTranslationMatrix[2][3] = -transformationMatrix[2][3];
            
    inverseTransformationMatrix = multiplyMatrices(inverseRotationMatrix, inverseTranslationMatrix);

    return inverseTransformationMatrix;

# For debugging, verify the product of a transformation matrix and its inverse is (very close to) the identity.
# inverseTransformationMatrix = invertTransformationMatrix(transformationMatrix);
# result = multiplyMatrices(transformationMatrix, inverseTransformationMatrix);
# print(result);

In [7]:
# 6: PRELIMINARY CODE INFRASTRUCTURE
# Code that can solve a 6×6 system of linear equations.
# Consulted the following resource: 

# x = Unknown intensities, Ax = b
def solveLinearEquations(_A, _b): 
    _x = np.linalg.solve(_A, _b)
    return _x;


In [8]:
# 1: ITERATIVE CLOSEST POINTS
# Read in two point clouds specified on the command line, 
# together with their associated rigid-body transforms (which are in separate files with the extension .xf). 
# For example, running
#     % icp file1.pts file2.pts
# should read in file1.pts, file1.xf, file2.pts, and file2.xf. 
# If either .xf file is missing, assume the identity transformation. 
# On output, your program will write file1.xf with the new transformation that aligns file1.pts to file2.pts.

In [9]:
# 2: ITERATIVE CLOSEST POINTS
# Randomly pick 1000 points on file1.

def randomlySamplePoints():
    file1Length              = len(open(file1PointsName, 'r').readlines(  ));
    file1IndexRange          = np.array(range(0, file1Length));
    file1PointsSampleIndices = np.random.choice(file1IndexRange, NUM_SAMPLES, replace=False)
    file1PointsSample        = np.ndarray( (NUM_SAMPLES, 6), float );

    for i, indexNumber in enumerate(file1PointsSampleIndices):
        file1PointsSample[i] = file1Points[indexNumber]
    
    return file1PointsSample

In [10]:
# 3: ITERATIVE CLOSEST POINTS
# For each point chosen in (2), apply M1, the current transformation of file1, 
# and then the inverse of M2, the transformation of file2. 
# This way, you have the location of the point in the coordinate system of file2. 
# Call these points pi.
def computePi(_M1, _inverseM2, _file1PointsSample):
    _pi = np.ndarray( (NUM_SAMPLES, 6), float );

    for i in range(0, NUM_SAMPLES):
        pt      = np.zeros(4)
        pt[0:3] = _file1PointsSample[i][0:3]
        pt[3]   = 1

        transformedPt_afterM1        = transformPt(       _M1, pt)
        transformedPt_afterInverseM2 = transformPt(_inverseM2, transformedPt_afterM1)
        
        _pi[i][0:3] = transformedPt_afterInverseM2[0:3]
        _pi[i][3:6] = _file1PointsSample[i][3:6]
    
    return _pi


In [11]:
# 4: ITERATIVE CLOSEST POINTS
# Find the closest point in file2 to each point computed in (3). 
# Call these points qi.

def computeQi(_pi):
    _qi = np.ndarray( (NUM_SAMPLES, 6), float );

    for i in range(0, NUM_SAMPLES):
        _qi[i] = findClosestPointKDTree(_pi[i], file2Points);
    
    return _qi

# The normal associated with each qi is ni.
def computeNi(_qi):
    _ni = np.ndarray( (NUM_SAMPLES, 3), float );

    for i in range(0, NUM_SAMPLES):
        _ni[i] = _qi[:][i][3:6]
    
    return _ni

# Debugging hint: Write out a .lines file containing your pi and qi.
# Use pts_view to load it up together with file1.pts and file2.pts.

In [12]:
# 5: ITERATIVE CLOSEST POINTS
# Compute the median point-to-plane distance between the 1000 point pairs. 
# That is, compute the median value of:  |(pi−qi)⋅ni|
def computePointToPlaneDistances(_pi, _qi, _ni): 
    _pointToPlaneDistances = np.zeros(NUM_SAMPLES);

    for i in range(0, NUM_SAMPLES):
        _pointToPlaneDistances[i] = np.abs( np.dot((_pi[:][i][0:3] - _qi[:][i][0:3]), _ni[:][i][0:3]) );
    
    return _pointToPlaneDistances

def computeMedianPointToPlaneDistance(_pointToPlaneDistances): 
    _medianPointToPlaneDistance = np.median( _pointToPlaneDistances ); 
    return _medianPointToPlaneDistance;

# Debugging hint: For the initial transformations provided to you, 
# you should be seeing initial median distances in the range of 1 through 10. The mesh units are millimeters.
# IT'S WORKING!

In [13]:
# 6: ITERATIVE CLOSEST POINTS
# To perform outlier rejection, eliminate (or mark as unused) any point pair whose 
# point-to-plane distance is more than 3 times the median distance you found in (5).
def computeOutlierIndices(_pointToPlaneDistances, _medianPointToPlaneDistance):
    _outlierIndices = []

    for i in range(0, NUM_SAMPLES):
        if np.abs( _pointToPlaneDistances[i] ) > 3 * _medianPointToPlaneDistance: 
            _outlierIndices.append(i)
            
    return _outlierIndices


In [14]:
# 7: ITERATIVE CLOSEST POINTS
# Compute the mean point-to-plane distance between the remaining point pairs found in (6). 
# This will be used later to see how much this ICP iteration reduced the misalignment.

def computeMeanPointToPlaneDistance(_pointToPlaneDistances, _outlierIndices): 
    sumPointToPlaneDistance = 0;
    numValidDistances = NUM_SAMPLES - len(_outlierIndices)

    for i in range(0, NUM_SAMPLES):
        if i not in _outlierIndices: 
            sumPointToPlaneDistance += np.abs( _pointToPlaneDistances[i] )
            
    _meanPointToPlaneDistance = (sumPointToPlaneDistance / numValidDistances)
    
    return _meanPointToPlaneDistance


In [15]:
# 8: ITERATIVE CLOSEST POINTS
# Construct the matrix C=ATA and the vector d=ATb. 
# You never actually need to construct the full matrix A or vector b, 
# if you construct C and d by summing up the appropriate contributions for each point pair.
#
# For each point i you will construct a 6×6 matrix and 6×1 vector, and sum up all of those matrices and vectors as C and d.
def computeCd(_pi, _qi, _ni, _outlierIndices):

    _C = np.ndarray( (6, 6), float );
    _d = np.ndarray( (6, 1), float );

    for x in range(0, 6): 
        _d.itemset((x, 0), 0)
        for y in range(0, 6): 
            _C.itemset((x, y), 0)

    # Calculate A transpose
    for i in range(0, NUM_SAMPLES):
        if i not in _outlierIndices: 
   
            pi_cross_ni = np.cross(_pi[i][0:3], _ni[i]);

            Ai      = np.zeros(6);
            Ai[0:3] = pi_cross_ni;
            Ai[3:6] = _ni[i];
            Ai_     = np.array(Ai)[np.newaxis]
            Ai_T    = Ai_.T

            bi = - np.dot( (_pi[:][i][0:3] - _qi[:][i][0:3]), _ni[:][i][0:3] )

            Ci = np.dot(Ai_T, Ai_);  # outer product
            _C = Ci + _C

            di = np.dot(Ai_T, bi);
            _d = di + _d
    
    dFlat = []
    for element in _d: 
        dFlat.append(element[0])

    return (_C, dFlat)

In [16]:
# 9: ITERATIVE CLOSEST POINTS
# Solve the system --> Cx=d <-- for the vector x, which consists of the rotations around the 3 axes 
# as its first 3 components, and the translation as its last 3 components. 
# xVector = solveLinearEquations(C, d);
# print(xVector);

def computeM_ICP(_xVector):
    rotationFactorX    = _xVector[0];
    rotationFactorY    = _xVector[1];
    rotationFactorZ    = _xVector[2];
    translationFactorX = _xVector[3];
    translationFactorY = _xVector[4];
    translationFactorZ = _xVector[5];

    # Construct a rigid-body transformation M_{ICP} out of those 6 values:
    # M_{ICP}=TRzRyRx 

    rotationXMatrix   = np.ndarray( (4, 4), float );
    rotationYMatrix   = np.ndarray( (4, 4), float );
    rotationZMatrix   = np.ndarray( (4, 4), float );
    translationMatrix = np.ndarray( (4, 4), float );
    
    # Referenced slides 56 & 57 (http://www.cs.princeton.edu/courses/archive/spr17/cos426/notes/cos426_s17_lecture10_transform.pdf)
    # to construct the basic translation (T) and rotation (Rx, Ry, Rz) matrices

    # Make an identity matrix as basis for all transformation matrices
    for x in range(0, 4): 
        for y in range(0, 4): 
            if x == y: 
                element = 1; 
            else: 
                element = 0;
            rotationXMatrix[x][y]   = element;
            rotationYMatrix[x][y]   = element;
            rotationZMatrix[x][y]   = element;
            translationMatrix[x][y] = element;

    rotationXMatrix[1][1] =   math.cos(rotationFactorX);
    rotationXMatrix[2][2] =   math.cos(rotationFactorX);
    rotationXMatrix[2][1] =   math.sin(rotationFactorX);
    rotationXMatrix[1][2] = - math.sin(rotationFactorX);

    rotationYMatrix[0][0] =   math.cos(rotationFactorY);
    rotationYMatrix[2][2] =   math.cos(rotationFactorY);
    rotationYMatrix[2][0] =   math.sin(-rotationFactorY);
    rotationYMatrix[0][2] = - math.sin(-rotationFactorY);

    rotationZMatrix[0][0] =   math.cos(rotationFactorZ);
    rotationZMatrix[1][1] =   math.cos(rotationFactorZ);
    rotationZMatrix[1][0] =   math.sin(rotationFactorZ);
    rotationZMatrix[0][1] = - math.sin(rotationFactorZ);

    translationMatrix[0][3] = translationFactorX;
    translationMatrix[1][3] = translationFactorY;
    translationMatrix[2][3] = translationFactorZ;

    Rx = rotationXMatrix;
    Ry = rotationYMatrix;
    Rz = rotationZMatrix;
    T  = translationMatrix;

    M_ICP = multiplyMatrices(multiplyMatrices(multiplyMatrices(T, Rz), Ry), Rx);
    
    return M_ICP

In [17]:
# 10: ITERATIVE CLOSEST POINTS
# Update the transformation of file1 to include the M_{ICP} you just computed.
# The tricky part is that M_{ICP} is expressed in the coordinate system of file2, 
# so the update that you need to perform is:
# M1 ← M2 M_{ICP} (M2)^-1 M1
# Make sure you understand why this is correct.

def updateTransform(_M2, _M_ICP, _inverseM2, _M1): 
    _transformedM1 = multiplyMatrices(multiplyMatrices(multiplyMatrices(_M2, _M_ICP), _inverseM2), _M1);
    return _transformedM1

In [18]:
# 11: ITERATIVE CLOSEST POINTS
# Update the pi in the list of points from (6) by multiplying them by M_{ICP}. 
# Re-compute the mean point-to-plane distance between point pairs, and compare to the distance you found in (7).

# Read in file1.pts, file2.pts
file1PointsName = file1Name + ".pts";
file2PointsName = file2Name + ".pts";
file1Points     = readPointCloud(file1PointsName);
file2Points     = readPointCloud(file2PointsName);

# Read in file1.xf, file2.xf
file1TransformationName   = file1Name + ".xf";
file2TransformationName   = file2Name + ".xf";
file1TransformationMatrix = readTransformation(file1TransformationName);
file2TransformationMatrix = readTransformation(file2TransformationName);

M1        = file1TransformationMatrix;
M2        = file2TransformationMatrix;
inverseM1 = invertTransformationMatrix(M1);
inverseM2 = invertTransformationMatrix(M2);

def doICP(_M1):
    
    file1PointsSample = randomlySamplePoints();

    pi = computePi(_M1, inverseM2, file1PointsSample);
    qi = computeQi(pi);
    ni = computeNi(qi);

    pointToPlaneDistances      = computePointToPlaneDistances( pi, qi, ni );
    medianPointToPlaneDistance = computeMedianPointToPlaneDistance( pointToPlaneDistances );
    outlierIndices = computeOutlierIndices(pointToPlaneDistances, medianPointToPlaneDistance);
    meanPointToPlaneDistance = computeMeanPointToPlaneDistance(pointToPlaneDistances, outlierIndices)
    
    print("Orig Median", medianPointToPlaneDistance);
    print("Orig Mean", meanPointToPlaneDistance)

    C, d    = computeCd(pi, qi, ni, outlierIndices);
    xVector = solveLinearEquations(C, d);
    M_ICP   = computeM_ICP(xVector)
    
    transformedM1 = updateTransform(M2, M_ICP, inverseM2, _M1);
    
    updatedPi = computePi(transformedM1, inverseM2, file1PointsSample);
    updatedQi = computeQi(updatedPi);
    updatedNi = computeNi(updatedQi);

    updatedPointToPlaneDistances      = computePointToPlaneDistances( updatedPi, updatedQi, updatedNi );
    updatedMedianPointToPlaneDistance = computeMedianPointToPlaneDistance( updatedPointToPlaneDistances );
    updatedOutlierIndices             = computeOutlierIndices( updatedPointToPlaneDistances, updatedMedianPointToPlaneDistance );
    updatedMeanPointToPlaneDistance   = computeMeanPointToPlaneDistance( updatedPointToPlaneDistances, updatedOutlierIndices );
    
    print("ICP Median", updatedMedianPointToPlaneDistance)
    print("ICP Mean", updatedMeanPointToPlaneDistance)
    print()
    
    return (updatedMeanPointToPlaneDistance / meanPointToPlaneDistance < 0.999), transformedM1


In [19]:
# 12: ITERATIVE CLOSEST POINTS
# If the ratio of distances found in (11) and (7) is less than 0.9999 
# (meaning that this ICP iteration has improved things by more than 0.01%), 
# continue with the next iteration of ICP by going back to step (2), else stop.

shouldUpdateAgain, updatedM1 = doICP(M1);

while shouldUpdateAgain:
    shouldUpdateAgain, updatedM1 = doICP(updatedM1);

Orig Median 0.20290159138510916
Orig Mean 0.14940632369158197
Orig Median 0.20290159138510916
Orig Mean 0.14940632369158197
Orig Median 0.20290159138510916
Orig Mean 0.14940632369158197
ICP Median 0.2019849662675418
ICP Mean 0.14880932815065848

ICP Median 0.2019849662675418
ICP Mean 0.14880932815065848

ICP Median 0.2019849662675418
ICP Mean 0.14880932815065848

Orig Median 0.1945841759019269
Orig Mean 0.15052121907239696
Orig Median 0.1945841759019269
Orig Mean 0.15052121907239696
Orig Median 0.1945841759019269
Orig Mean 0.15052121907239696
ICP Median 0.19777426534944537
ICP Mean 0.1476273757268138

ICP Median 0.19777426534944537
ICP Mean 0.1476273757268138

ICP Median 0.19777426534944537
ICP Mean 0.1476273757268138

Orig Median 0.2282552644080363
Orig Mean 0.16529745321543468
Orig Median 0.2282552644080363
Orig Mean 0.16529745321543468
Orig Median 0.2282552644080363
Orig Mean 0.16529745321543468
ICP Median 0.22341228071535396
ICP Mean 0.1616849371275964

ICP Median 0.223412280715353

In [20]:
# 13: ITERATIVE CLOSEST POINTS
# Write out the new M1 in file1.xf
file = open(file1TransformationName, 'w')

for x in range(0, 4): 
    for y in range(0, 4): 
        file.write(str(updatedM1[x][y]) + ' ') 
    file.write(' \n') 

file.close()